In [4]:

!pip install -U joblib huggingface_hub scikit-learn
!pip install -U datasets
import re
import nltk
import string
import pandas as pd
from google.colab import files
from datasets import load_dataset
from nltk.corpus import stopwords

from huggingface_hub import hf_hub_download
import joblib
nltk.download('stopwords', quiet=True)
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer


uploaded = files.upload()
df_demo = pd.read_csv('demo.csv', encoding='utf-8')


lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation + '0123456789'))
    text = text.replace('“', '"').replace('”', '"').replace('‘', "'").replace('’', "'")
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

stop_words = set(stopwords.words('english'))

df_demo['text'] = df_demo['text'].apply(preprocess)
df_demo['text'] = df_demo['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))



# Load model from huggingface
repo_id = "nnvy/svm-climate"
model_path = hf_hub_download(repo_id=repo_id, filename="modelv2.joblib")
vectorizer_path = hf_hub_download(repo_id=repo_id, filename="vectorizerv2.joblib")


model = joblib.load(model_path)
vectorizer = joblib.load(vectorizer_path)


X_demo = vectorizer.transform(df_demo['text'])
demo_preds = model.predict(X_demo)

df_preds = pd.DataFrame(demo_preds, columns=["prediction"])

df_preds.to_csv("svm_predictions.csv", index=False)

files.download("svm_predictions.csv")
print("Predictions:", demo_preds)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Saving demo.csv to demo (2).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Predictions: [1 2 1 1 1 0]
